# HW 7:
This homework we are going to start with a dataset ingest, some cleaning and some visualizations. Then move to Streamlit

## US Census Data

Load the data posted on the github repo "us-population-2010-2019.csv"

In [1]:
# Your code here:
import pandas as pd 

In [2]:
data = pd.read_csv("us-population-2010-2019-states-code.csv")

Using a lambda function, add a column to the dataframe that provides standard two letter abbreviations to all of the US States. For example, Connecticut would be CT.
https://www.50states.com/abbreviations.htm
*hint* there is a python package called "us"

In [3]:
#Your code here
!pip install us

In [4]:
import us

In [5]:
state_col=data.states # took "states" column and named state_col
#pd.wide_to_long(data, ["state", "state_abbr"], i='id', j='year')
#df_reshaped['year']=pd.to_numeric(df_reshaped['year'])

In [6]:
state_abbr = [] # list of state abbreviations 
for each in us.STATES: # "each" is each state
    state_abbr.append(each.abbr) # converted state to state abbreviation and appended to list 

In [7]:
state_abbr.append('PR')
state_abbr.insert(8, 'DC') # added two more that exist in the census dataset 

In [8]:
data['state_abbr']=state_abbr # added to the original dataset 
#data

In [9]:
a=data.iloc[:,3:13] # extracted year columns 

In [10]:
df_reshaped=data.melt(id_vars=['id', 'states', 'state_abbr'], # reshaped so that year is one column 
                      value_vars=a, var_name='year', value_name='population')
#df_reshaped

Reshape your data, and create a new df called df_reshaped so that:

1. Convert 'year' column values to integers
2. Convert 'states' to strings
3. Get rid of the commas in the population numbers, and convert them to integers
4. Check your df_reshaped 

In [11]:
# Your code here:
df_reshaped.dtypes # checked data types 

id             int64
states        object
state_abbr    object
year          object
population    object
dtype: object

In [12]:
df_reshaped['year']=df_reshaped['year'].astype(int) # convert "year" to integer 
df_reshaped['states']=df_reshaped['states'].astype(str) # convert "states" to string 

In [13]:
def remove_comma(col): # remove commas from population 
    return col.str.replace(',', '') 

In [14]:
df_reshaped['population']=remove_comma(df_reshaped['population']) # remove comma using the function
df_reshaped['population']=df_reshaped['population'].astype(int) # convert to int type 

Save your df_reshaped to a csv file

In [15]:
#Your code here:
df_reshaped.to_csv("reshaped_data.csv")

Subset your dataframe by selected_year = 2019 year create a new dataframe called df_selected_year


In [16]:
#Your code here:
df_selected_year = df_reshaped[df_reshaped['year']==2019] # filter for 2019 

Sort df_selected_year by population, from highest to lowest, and create a new df called "df_selected_year_sorted"

In [17]:
#Your code here:
df_selected_year_sorted = df_selected_year.sort_values(by = ['population'], ascending= False)

In [18]:
# Here is a function to calculate population difference between selected and previous year
def calculate_population_difference(input_df, input_year):
  selected_year_data = input_df[input_df['year'] == input_year].reset_index()
  previous_year_data = input_df[input_df['year'] == input_year - 1].reset_index()
  selected_year_data['population_difference'] = selected_year_data.population.sub(previous_year_data.population, fill_value=0)
  selected_year_data['population_difference_absolute'] = abs(selected_year_data['population_difference'])
  return pd.concat([selected_year_data.states, selected_year_data.id, selected_year_data.population, selected_year_data.population_difference, selected_year_data.population_difference_absolute], axis=1).sort_values(by="population_difference", ascending=False)

df_population_difference_sorted = calculate_population_difference(df_reshaped, 2019)
#df_population_difference_sorted

In [19]:
# Filter states with population difference > 50000
df_greater_50000 = df_population_difference_sorted[
    df_population_difference_sorted['population_difference_absolute']>50000]

In [20]:
#df_greater_50000

In [21]:
# Calculate the % of States with population difference > 50000
#Your code here:
perct = round(len(df_greater_50000)/len(data)*100, 2)
print(f"{perct}% of states have population difference > 50000 in 2019.")

26.92% of states have population difference > 50000 in 2019.


## Plots

### Heatmap: run the following code to see a heatmap

In [22]:
!pip install altair
import altair as alt

alt.themes.enable("dark")

heatmap = alt.Chart(df_reshaped).mark_rect().encode(
        y=alt.Y('year:O', axis=alt.Axis(title="Year", titleFontSize=16, titlePadding=15, titleFontWeight=900, labelAngle=0)),
        x=alt.X('states:O', axis=alt.Axis(title="States", titleFontSize=16, titlePadding=15, titleFontWeight=900)),
        color=alt.Color('max(population):Q',
                         legend=alt.Legend(title=" "),
                         scale=alt.Scale(scheme="blueorange")),
        stroke=alt.value('black'),
        strokeWidth=alt.value(0.25),
        #tooltip=[
        #    alt.Tooltip('year:O', title='Year'),
        #    alt.Tooltip('population:Q', title='Population')
        #]
    ).properties(width=900
    #).configure_legend(orient='bottom', titleFontSize=16, labelFontSize=14, titlePadding=0
    #).configure_axisX(labelFontSize=14)
    ).configure_axis(
    labelFontSize=12,
    titleFontSize=12
    )

heatmap

alt.Chart(...)

### Choropleth: Run the following code to get a map of the population for the selected year above

In [23]:
# Choropleth via Altair
!pip install vega_datasets
import altair as alt
from vega_datasets import data

alt.themes.enable("dark")

states = alt.topo_feature(data.us_10m.url, 'states')

alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('population:Q', scale=alt.Scale(scheme='blues')),   # scale=color_scale
    stroke=alt.value('#154360')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_selected_year, 'id', list(df_selected_year.columns))
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)



alt.Chart(...)

Pycharm:
Create a pdf reader streamlit app in pycharm using the following code.Upload some pdfs and perform some queries. 
Below capture 5 queries and 5 responses. Check to make sure the responses are accurate. See if you can get your app to fail. Analyze why.

In [24]:
#!pip install streamlit

In [25]:
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain



def get_pdf_text(pdf_docs):

    text: str = ''
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
           text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks= text_splitter.split_text(text)
    return chunks

def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

def get_conversation_chain(vectorstore):
    llm = ChatOpenAI()
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

def handle_userinput(user_question):
    response = st.session_state.conversation({'question':user_question})
    st.session_state.chat_history = response['chat_history']
    for i, message in enumerate(st.session_state.chat_history):
        if i%2 ==0:
            st.write(f'Human Question: {message.content}')
        else:
            st.write(f'AI Response: {message.content}')

        
def main():
    load_dotenv()
    st.set_page_config(page_title="Chat with PDFs", page_icon=":scalpel:")

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history= None

    st.header("Chat with PDFs :medical_symbol:")
    user_question = st.text_input("Ask a question about your documents:")
    if user_question:
        handle_userinput(user_question)

    with st.sidebar:
        st.subheader("Your documents")
        pdf_docs = st.file_uploader("Upload your PDFs here and click on 'Process'", accept_multiple_files=True)
        if st.button('Process'):
            with st.spinner("Processing..."):

            #get pdf texts
                raw_text = get_pdf_text(pdf_docs)
                #st.write(raw_text)

            #chunk pdfs
                text_chunks = get_text_chunks(raw_text)
                #st.write(text_chunks)

            #create vector store with embeddings
                vectorstore = get_vectorstore(text_chunks)

            # create conversation chain
                st.session_state.conversation = get_conversation_chain(vectorstore)


if __name__=='__main__':
    main() 

2024-04-01 23:24:24.446 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-04-01 23:24:24.708 
  command:

    streamlit run /Users/jiyonglee/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


1. On the article titled "A government proposal to kill a half-million owls sparks controversy"
- Query: What's the article about?
- Answer: "The article is about a proposal by the U.S. Fish and Wildlife Service to address the threat posed by barred owls to the northern spotted owl population. The proposal involves killing barred owls to save the spotted owls, as barred owls have encroached on the spotted owls' territory due to human-driven habitat destruction. Conservation experts supporting the proposal see it as a necessary plan to solve a human-created problem and prevent the extinction of the northern spotted owl. The public comment period for the proposal has closed, and a final decision is expected in the spring or summer."

2. On the same article: 
- Human Question: Why do some people oppose the proposal?
- "Some people, like Wayne Pacelle, president of the Center for a Human Economy and Animal Wellness Action, oppose the proposal because they believe it is unsustainable and doubt that it can be successfully executed over such a long time frame. They argue that a management plan covering a vast region like this wouldn't be as effective as successful lethal management programs implemented in closed ecosystems."

3.
- Are there any examples on a similar case in the past?
- "Yes, there have been similar conservation proposals in the past where one species is targeted to protect another. An example mentioned in the provided context is a four-year experiment conducted by the Fish and Wildlife Service a decade ago to kill up to 3,600 barred owls in the northwest to save the spotted owl. While this strategy slowed down the barred owl invasion, it did not completely stop it."

4.
- Is the writer of this article agreeing with the proposal?
- "The writer of the article does not explicitly state their personal opinion on the proposal to kill barred owls to save northern spotted owls. The article presents various perspectives from different experts, conservation organizations, and advocates without clearly indicating the writer's personal stance on the matter."

5.
- Are there any examples on a similar case of animal conservation from the past?
- "I don't know."

Your analysis here:
- My last query returned "I don't know" possibly because it was not possible to answer just from the pdf file. The answer to the third query is something directly from the article (as expected). 
- Other queries and answers show that it is good at summarizing the content of pdf. 
- My fourth query might be tricky for other articles if the writer's intention is more nuanced.